# Initalize libraries

## Import libraries

In [ ]:
import sys, os
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
from tqdm.auto import tqdm

#data
import xarray as xr
import h5py


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm

# Open nexus files
from nexusformat.nexus import *

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
import helper_functions as helper
import interactive
from interactive import cimshow
import mask_lib

# Facility specific loading functions
import MAXI_loading as loading

# Gifs
import imageio

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
#%load_ext jupyter_black

In [ ]:
PROPOSAL = 11018955
USER = getuser()

BASEFOLDER = "/data/export/cklose/2409_MAXIV"#"/asap3/petra3/gpfs/p04/2024/data/%s/" % PROPOSAL
sample_name = "2408_"
spefile_name = "scan"

# Load mnemonics for easy data handling
mnemonics = loading.mnemonics

### Loading data

In [ ]:
def generate_filename(scan_nr,file_format=".h5"):
    """
    Generates filename of the given scan id
    
    Parameter
    =========
    file_format : str
        file format (ending, e.g. ".nxs")
    scan_nr : int or str
        number identifier (id) of the given scan
        
    Output
    ======
    filename : str
        full generated filename
    ======
    author: ck 2024
    """

    # Convert scan number to string
    if type(scan_nr) == int:
        scan_nr = "%d" % scan_nr
    elif isinstance(scan_nr, np.generic):
        scan_nr = "%d" % scan_nr
        
    # Combine all inputs
    filename = join(BASEFOLDER,"data","raw",sample_name+scan_nr+file_format)
    
    return filename

def load_data(scan_id, keypath = mnemonics["pre_scan_snapshot"], keys = None):
    """
    Load data of all specified keys from keypath
    
    Parameter
    =========
    scan_id : int
        experimental identifier of scan
    keypath : str
        path of nexus file tree to relevant data field
    keys : str or list of strings
        keys to load from keypath
        
    Output
    ======
    data : dict
        data dictionary of keys
    ======
    author: ck 2024
    """
        
    #Generate filename from scan_id
    fname = generate_filename(scan_id)
    
    # load data with basic loading function
    data = loading.load_data(fname, keypath, keys = keys)
    
    return data


def load_key(scan_id, key):
    """
    Load any kind of data specified by key (path)
    
    Parameter
    =========
    scan_id : int
        experimental identifier of scan
    key : str
        key path of nexus file tree to relevant data field
   
    Output
    ======
    data : dict
        data dictionaray on single key
    ======
    author: ck 2024
    """
    #Generate filename from scan_id
    fname = generate_filename(scan_id)
    
    # load data with basic loading function
    data = loading.load_key(fname, key)
    
    return data

### Loading images

In [ ]:
def get_filepath(tiffname, camera_type = "cmos"):
    """Return raw data path from save file path."""
    head, tifffile = path.split(tiffname)
    head, scandir = path.split(head)
    return path.join(BASEFOLDER,"data","raw", camera_type, scandir, tifffile)

def load_tiff(tifffile):
    """load single tiff image into np array"""
    return np.array(Image.open(tifffile))

def load_tiff_list(filelist, processes=None):
    """Multiprocessing loading of image files"""
    with Pool(processes=processes) as pool:
        frames = pool.map(load_tiff, filelist)
    return np.stack(frames)

def load_cmos(scanid):
    # Load averaged image if it does exist
    fname = join(BASEFOLDER,"data","raw","cmos", "scan_%04d"%scanid, "scan_%04d_avg.h5"%scanid)

    if os.path.exists(fname):        
        print("Loading: %s"%fname)
        with h5py.File(fname, "r") as f:
            # Get entry
            image = f["image"][()].squeeze()
    
        return image

    # Loads all single frames
    else:
        print("Loading single frames")
        fname = generate_filename(scanid,".h5")
        cmos_filelist = load_key(scanid,mnemonics["cmos"])
        cmos_filelist = [get_filepath(b.decode(),camera_type = "cmos") for b in cmos_filelist]
        
    return load_tiff_list(cmos_filelist)

def load_images(im_id, camera_type = "ccd"):
    """
    Loads all images of a given image id
    
    Parameter
    =========
    im_id : int
        experimental identifier of image scan
    camera_type : str
        "ccd" or "cmos", select corresponding kezy from mnemonics dict
   
    Output
    ======
    images : array
        stack of all images the correspond to the id
    ======
    author: ck 2024
    """

    if camera_type == "ccd":
        # Find spe files
        spe_file_list = [get_filepath(b.decode(),camera_type = "ccd") for b in cmos_filelist]
        
        # Loop over image spe files
        images = []
        for spe_file in spe_file_list:
            images.append(np.squeeze(np.array(imageio.mimread(spe_file, memtest="5000MB"))))

        # Stack to 3d array
        images = np.stack(images)
    elif camera_type == "cmos":
        images = load_cmos(im_id)

    return images

### Loading image procedure

In [ ]:
# Full image loading procedure
def load_processing(im_id, crop=None,camera_type = "cmos"):
    """
    Loads images, averaging of two individual images (scans in tango consist of two images),
    padding to square shape, Additional cropping (optional)
    """

    # Load data
    images = load_images(im_id,camera_type=camera_type)

    # Zeropad to get square shape
    #images = helper.padding(images)

    # Calculate mean
    if images.ndim > 2:
        image = np.mean(images, axis=0)
    else:
        image = images.copy()

    # Optional cropping
    if crop is not None:
        images = images[:, :crop, :crop]
        image = image[:crop, :crop]

    return image, images

# Experimental Details

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.09,  # ccd to sample distance
    "px_size": 13.5e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)

# General saving folder
folder_general = helper.create_folder(join(BASEFOLDER, "processed"))
print("Output Folder: %s" % folder_general)

# Load images


In [ ]:
# Specify image ids in iterable like list or array
im_ids = [332,333]
dark_id = 331*np.ones(len(im_ids)) #iterable; one needs to be assigned for each image

# Which data from nexus files to load? (e.g. "energy","srotz", ...)
key = "energy"

## Normalize?
normalization = False
norm_key = "maxibeckhoff1adc2_rmean"


# Load data
images, scan_axis, norm = [], [], []
for i, im_id in enumerate(tqdm(im_ids)):
    # load images
    _, timages = load_processing(im_id)

    ## Load and subtract dark images
    if dark_id is not None:
        dark, _ = load_processing(dark_id[i])
        timages = timages - dark
    
    # Append to data variable
    images.append(timages)

    # Load scan axis or single image data
    try:
        scan_axis.append(load_data(im_id, mnemonics["measurement"])[key])
    except:
        scan_axis.append(load_data(im_id, mnemonics["pre_scan_snapshot"])[key])
    
    # Load normalization
    if normalization is True:
        norm.append(load_data(im_id, mnemonics["measurement"])[norm_key])
    else:
        norm.append(np.ones(timages.shape[0]))

# Get one lambda value
if key == "energy":
    energy_lambda = cci.photon_energy_wavelength(scan_axis[0])
else: 
    energy_lambda = cci.photon_energy_wavelength(load_key(im_id,mnemonics["energy"]))

# Make it beautiful
try:
    scan_axis = np.concatenate(scan_axis)
except:
    scan_axis = np.stack(scan_axis)
  
try:
    norm = np.concatenate(norm)
except:
    norm = np.stack(norm)

images = np.stack(images)

    
# Squeeze list
images = np.squeeze(images)
scan_axis = np.squeeze(scan_axis)
norm = np.squeeze(norm)

# Plot scan axis
fig, ax = plt.subplots()
ax.plot(scan_axis, "o-")
ax.set_title("Scan Axis")
ax.set_ylabel(key)
ax.set_xlabel("Frame Index")
ax.grid()

In [ ]:
cimshow(images)

In [ ]:
# Assign to xarray
# Setup xarray for images
data = xr.Dataset()
data[key] = xr.DataArray(scan_axis, dims=["index"])
# data["norm"] = xr.DataArray(norm, dims=["index"])
data["images"] = xr.DataArray(images, dims=["index", "y", "x"])

data["image"] = data["images"].mean("index")
image = data["image"].values

# Sort ascending key
data = data.sortby(key)

# Assign im_id as attribute
data = data.assign_attrs({"im_id": im_id})

# Slideshow viewer
fig, ax = cimshow(data["images"])

# Draw beamstop mask

In [ ]:
poly_mask = interactive.draw_polygon_mask(image)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.get_vertice_coordinates()
mask_draw = poly_mask.full_mask.astype(int)

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
tmp = image * (1 - mask_draw)
mi, ma = np.percentile(tmp[tmp != 0], [0.1, 99.9])
ax[0].imshow(image * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(image * mask_draw, [0.1, 99.9])
ax[1].imshow(image * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")

In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Setup dictonary
    mask_coordinates = dict()

    # Mask #1
    mask_coordinates["bs_medium"] = [[(972.9776892552021, 1004.2743085178313), (683.0854056548453, 1011.0191797381575), (429.3488120641165, 1014.0243786386102), (-5.38786287953252, 1010.0262182986162), (-11.784588413299048, 1027.546033470307), (-6.456709339024059, 1050.189519535976), (407.3336253818711, 1051.9815970562745), (785.5107592764842, 1048.7548363356034), (970.4099189292948, 1041.9313069708498), (977.0697677721386, 1049.9231255822624), (987.7255259206885, 1067.238732573656), (1005.041132912082, 1075.2305511850686), (1010.3690119863572, 1093.8781279450309), (1022.57787485063, 1452.2001151306752), (1023.891500713352, 1538.8489667863084), (1029.7269726228894, 1668.2234921518748), (1036.1520087654237, 1833.2762608301566), (1039.0507610028706, 2010.5397226740427), (1041.0931146480093, 2061.6873617870824), (1083.7161472422092, 2055.027512944239), (1082.3841774736404, 1997.7528128957829), (1061.329837800928, 1535.5648530012597), (1045.621811861143, 1081.8904000279126), (1059.0201832634914, 1068.6419757912079), (1080.0381970670428, 1045.6535231935734), (1250.4043869753773, 1036.9986877096228), (1711.4432772620323, 1003.3274407611157), (1930.818796336602, 981.3023122296238), (1968.2328715564279, 975.4216165269843), (2057.474846050534, 986.0773746755342), (2052.9860015697454, 945.8344154978255), (1975.482075669149, 939.2662861903159), (1901.4373645403866, 946.447411617374), (1619.8397671015532, 973.7708588493829), (1382.0734859488762, 992.1616209292713), (1192.6712743111323, 1001.972213913788), (1074.126880684794, 1006.2447473119142), (1066.1341462971013, 991.3164557652381), (1042.158690462864, 972.6688790052756), (1026.505884395897, 560.1120440507323), (1007.553206285446, -9.489556366445356), (970.9641323120604, -5.802018358814962), (976.3933883688477, 150.2045104582064), (980.8479680128388, 277.27966418395783), (990.3028627366012, 506.77436338214807), (997.1844288881059, 749.2817066564569), (1006.4751487546124, 969.463223155699), (984.8003220196999, 986.5403593710846)]]
    mask_coordinates["bs_large"] =[[(-9.52514267623522, 988.0244972208288), (133.8623449984891, 1003.0301645356255), (444.7643270650601, 1004.8352658735243), (677.4009610677929, 1006.3647572722471), (999.1891601517673, 998.0282754306933), (1008.9687632034429, 924.746257259271), (1002.856550157784, 753.6042842087398), (990.8526783102134, 392.799693733892), (977.5143073637275, -17.355212870551895), (1009.1929383616316, -5.684138292376701), (1027.5331984130498, 374.4594336824738), (1040.871569359536, 736.2627456059059), (1047.540754832779, 842.9697131777938), (1049.2080512010896, 983.0226081158966), (1113.8991502915467, 991.4702430486711), (1282.2960834909322, 986.4683539437389), (1529.0559460009226, 968.1280938923206), (1872.5189978729368, 933.1148701577949), (1955.8838162884742, 928.1129810528627), (2022.575671020904, 939.784055631038), (2029.2448564941471, 996.4721321536034), (1947.5473344469203, 974.7972793655637), (1627.4264317312568, 998.139428521914), (1333.9822709085652, 1019.8142813099539), (1133.9067067112755, 1031.485355888129), (955.5059953020256, 1034.8199486247506), (921.8266086621485, 1036.8207042667236), (678.7348313316545, 1045.0328414964195), (469.98929284993585, 1046.824482476588), (116.52246276805741, 1046.824482476588), (-6.8574684869379325, 1023.4823333202376)], [(-18.49404559764608, 785.86607528193), (-15.741149396111126, -20.73251176780832), (1721.3363537724376, -15.226719364738301), (2061.961377109028, -16.878457085658965), (2070.998269815253, 2083.925571530775), (-33.65294626451481, 2100.746691469454), (-41.88066798492048, 1173.7567109934555), (-50.10838970532615, 731.8366354770906), (43.139123125938, 737.3217832910489), (81.53515782116438, 62.64860221779031), (960.9090655017594, 46.10727357274732), (1962.9632828604786, 59.87175458042202), (2008.0241868014546, 946.3583113396406), (1981.6954772961565, 1969.1555403163811), (838.0421581597698, 2002.0664271980036), (56.04291820032506, 1965.3159367546796), (34.102326945909965, 1019.127939000208), (36.84490085271187, 742.1279744132175)]]
    
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load? Use can combine multiple masks, e.g., ["bs_left_part", "bs_bot_part", "bs_top_part"]
polygon_names = ["bs_large"]
mask = mask_lib.load_poly_masks(
    experimental_setup["binning"] * np.array(image.shape),
    load_poly_coordinates(),
    polygon_names,
)

# optional binning
mask = helper.binning(mask,experimental_setup["binning"])

# Move beamstop
mask = cci.shift_image(mask, [-12, 80])
mask = mask + cci.circle_mask(mask.shape,[1000,1113],100)

mask[:,:82] = 1 #broken detector row
mask[mask>1]=1
mask = np.round(mask)
mask = mask.astype(int)

# Add to xarray
data["mask"] = xr.DataArray(mask, dims=["y", "x"])

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(image, [0.1, 99.9])
ax[0].imshow(image, cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask)")

mi, ma = np.percentile(image * mask, [0.1, 99.9])
ax[1].imshow(image * mask, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask")

ax[2].imshow(1 - mask)
ax[2].set_title("1 - mask")
plt.tight_layout()

## Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. 

In [ ]:
# Set center position via widget
c0, c1 = 1022, 1090  # initial values
ic = interactive.InteractiveCenter(data["images"], c0=c0, c1=c1)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

## Azimuthal integrator widget for finetuning
If scattering pattern is radial symmetric, move center position until scattering ring is a line after transformation in polar coordinates

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=energy_lambda,#experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Calc azimuthal integration
I_t, q_t, phi_t = ai.integrate2d(
    image,
    200,
    radial_range=(0.025, 1.3),
    unit="q_nm^-1",
    correctSolidAngle=True,
    dummy=np.nan,
    mask=mask,
    method = "BBox"
)
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.nanpercentile(I_t, [1, 98])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

In [ ]:
aic = interactive.AzimuthalIntegrationCenter(
    image * (2 - mask),
    ai,
    c0=center[0],
    c1=center[1],
    im_data_range=[1, 95],
    radial_range=(0.025, 0.5),
    qlines=[40, 60],
    mask=mask,
    circle_radius=100,
)

In [ ]:
# Get center positions
center = [aic.c0, aic.c1]
data = data.assign_attrs({"center": center})
print(f"Center:", center)

# Azimuthal integration for all images

In [ ]:
# Update center of azimuthal integrator
ai = AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=energy_lambda,
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Do 2d Azimuthal integration of all images and append them to list
list_q, list_i2d = [], []
for i, im in enumerate(tqdm(data["images"].values)):
    # Adapt azimuthal integrator if scan is an energy scan
    if key == "energy":
        ai.wavelength = cci.photon_energy_wavelength(
            data["energy"][i].values, input_unit="eV"
        )

    # Calc ai
    i2d, q, chi = ai.integrate2d(
        im,
        200,
        90,
        radial_range=(0.0, 2),
        unit="q_nm^-1",
        correctSolidAngle=True,
        dummy=np.nan,
        mask=mask,
        method = "BBox"
    )
    list_q.append(q)
    list_i2d.append(i2d)

# Add to xarrays
data["q"] = q
data["chi"] = chi
data["i2d"] = xr.DataArray(list_i2d, dims=["index", "chi", "q"])

## Select relevant chi-range

In [ ]:
# Plot 2d and 1d azimuthal integration to estimate the relevant chi and q range
# which image to show?
idx = 0

# Select chi-range
# Which chi-mode? ("all","other")
chi_mode = "all"

# Select chi-range
if chi_mode == "all":
    sel_chi = (data.chi <= 180) * (data.chi >= -180)
elif chi_mode == "other":
    sel_chi = (
        (data.chi <= 139) * (data.chi >= 132)
        + (data.chi <= 47) * (data.chi >= 41)
        + (data.chi <= -40) * (data.chi >= -48)
        + (data.chi <= -133) * (data.chi >= -140)
    )
data["i1d"] = data.i2d.where(sel_chi, drop=True).mean("chi")

# Plot
fig, ax = plt.subplots(
    2,
    1,
    figsize=(8, 8),
    sharex=True,
)
mi, ma = np.nanpercentile(data["i2d"][idx], [0.1, 99.9])
data["i2d"][idx].plot.imshow(ax=ax[0], vmin=mi, vmax=ma)
ax[0].set_title(f"2d Azimuthal integration")
ax[0].grid()

# Plot 1d azimuthal integration to estimate the relevant q-range
ax[1].plot(data.q, data.i1d[idx])
ax[1].set_yscale("log")
ax[1].set_title("1d Azimuthal Integration")
ax[1].grid()
ax[1].set_ylabel("Integrated intensity")
ax[1].set_xlabel("q")

## Select relevant q-range

In [ ]:
# Select relevant q-range for averaging
q0, q1 = 0, 0.75
binning = False
bins = []

# Get SAXS from q-range
sel = (data.q > q0) * (data.q < q1)

data["saxs"] = data.i1d.where(sel, drop=True).mean("q")

# Averaging of same scan axis values or binning
if binning is True:
    # Execute binning
    data_bin = data.groupby_bins(key, bins).mean()

    # Rename binned values, drop intervals as those cannot be save in h5
    bin_scan_axis = scan_axis + "_bins"
    data_bin = data_bin.swap_dims({bin_scan_axis: key})
    data_bin = data_bin.drop(bin_scan_axis)
else:
    _, count = np.unique(data[key].values, return_counts=True)
    if np.any(count > 1):
        data_bin = data.groupby(key).mean()
    else:
        data_bin = data.swap_dims({"index": key})

In [ ]:
# Plot Averaged 1d Intensity over q for all different scan 
fig, ax = plt.subplots()
colors = plt.cm.jet(np.linspace(0,1,len(data_bin[key])))

for i, i1dchi in enumerate(data_bin["i1d"].values):
    #i1dchi = i1dchi - np.nanmin(i1dchi)
    ax.plot(data_bin["q"],i1dchi/np.nanmax(i1dchi),label="%s: %.1f"%(key,data_bin[key][i]),color = colors[i])
    
ax.grid()
ax.legend(ncol = 2,fontsize = 8)
ax.set_xlabel("q")
ax.set_ylabel("Averaged Intensity")

## Title and fname
if len(im_ids) > 1:
    ax.set_title("Scan Id %s-%s" % (im_ids[0], im_ids[-1]))
    fname = "SAXS_i1d_ImId_%04d-%04d_%s.png" % (im_ids[0], im_ids[-1], USER)
else:
    ax.set_title("Scan Id %d" % (im_ids[0]))
    fname = "SAXS_i1d_ImId_%04d_%s.png" % (im_ids[0], USER)

fname = join(folder_general, fname)
print("Saving:%s" % fname)
plt.savefig(fname)

# Plotting

In [ ]:
# Plot Intensity of SAXS Pattern
fig, ax = plt.subplots()
ax.plot(data_bin[key].values, data_bin["saxs"].values)
ax.grid()
ax.set_xlabel(key)
ax.set_ylabel("Integrated SAXS")

## Title and fname
if len(im_ids) > 1:
    ax.set_title("Scan Id %s-%s" % (im_ids[0], im_ids[-1]))
    fname = "SAXS_ImId_%04d-%04d_%s.png" % (im_ids[0], im_ids[-1], USER)
else:
    ax.set_title("Scan Id %d" % (im_ids[0]))
    fname = "SAXS_ImId_%04d_%s.png" % (im_ids[0], USER)

fname = join(folder_general, fname)
print("Saving:%s" % fname)
plt.savefig(fname)

In [ ]:
# Plot I(q,t) and integrated intensity
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6, 4.5), sharex=True)

# Create log plot
data_bin["i1dlog"] = np.log10(data_bin["i1d"]+ 1)

#vmin, vmax = data_bin["i1d"].min(), data_bin["i1d"].max()
vmin, vmax = np.nanpercentile(data_bin["i1dlog"], [1, 99])
data_bin["i1dlog"].plot.contourf(
    x=key,
    y="q",
    ax=ax[0],
    cmap="viridis",
    add_colorbar=False,
    vmin=vmin,
    vmax=vmax,
    levels=100,
    ylim = [0,q1]
)

ax[1].plot(data_bin[key], data_bin["saxs"], "o-")
ax[1].grid()
ax[1].set_ylabel("total scattered intensity")
ax[1].set_xlabel(key)

fig.suptitle(f"ImId:{im_ids}")

fname = join(folder_general, "SAXS_ImId_%04d_%s.png" % (im_ids[0], USER))
print("Saving: %s" % fname)
plt.savefig(fname)

# Export scan as gif

## Select roi of images for plotting

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(data_bin["images"].values)

In [ ]:
# Takes start and end of x and y axis
roi = interactive.axis_to_roi(ax)
print(f"Image registration roi:", roi)

## Plotting

In [ ]:
# Setup gif
folder_gif = helper.create_folder(join(folder_general, "ImId_%05d" % im_id))
variable_images_1d = []

# Find global max and min all images
allmin, allmax = np.nanpercentile(data_bin["i2d"].values, [0.1, 100])
allImin = data_bin.i1d.where(sel, drop=True).min()
allImax = data_bin.i1d.where(sel, drop=True).max()

#if allmin < 5:
#    allmin = 5

# Loop over images
for i in tqdm(range(len(data_bin[key].values))):
    # Plot for averaged image
    fig = plt.figure(figsize=(6, 10))
    gs1 = gridspec.GridSpec(
        4,
        1,
        figure=fig,
        left=0.2,
        bottom=0.05,
        right=0.975,
        top=1.1,
        wspace=0,
        hspace=0,
        height_ratios=[6, 1, 2, 1],
    )

    # Plot image roi
    ax0 = fig.add_subplot(gs1[0])
    m = ax0.imshow(data_bin["images"][i].values[roi], vmin=allmin, vmax=allmax)
    plt.colorbar(m, ax=ax0, pad=0.045, location="bottom")

    # Plot 1d azimuthal integration
    ax1 = fig.add_subplot(gs1[1])
    tmp = data_bin.i1d[i]
    ax1.plot(data_bin.q, tmp)
    ax1.set_xlabel("q")
    ax1.set_ylabel("Mean Intensity")
    ax1.set_xlim([.1, q1])
    ax1.set_ylim([allmin, allImax])
    ax1.set_yscale("log")
    ax1.grid()
    
    ax2 = fig.add_subplot(gs1[2])
    vmin, vmax = np.nanpercentile(data_bin["i1dlog"], [.1, 99.9])
    data_bin["i1dlog"].plot.contourf(
        x=key,
        y="q",
        ax=ax2,
        cmap="viridis",
        add_colorbar=False,
        vmin=vmin,
        vmax=vmax,
        levels=200,
        ylim = [.1,q1]
    )
    ax2.vlines(data_bin[key].values[i], 0.1, q1,'r')
    ax2.hlines(q0, data_bin[key].min(),data_bin[key].max(),'w',linestyles='dashed')
    ax2.hlines(q1, data_bin[key].min(),data_bin[key].max(),'w',linestyles='dashed')

    # Plot SAXS Intensity
    ax3 = fig.add_subplot(gs1[3])
    ax3.plot(data_bin[key].values, data_bin["saxs"].values)
    ax3.scatter(data_bin[key].values[i], data_bin["saxs"].values[i], 20, color="r")
    ax3.set_xlabel(key)
    ax3.set_ylabel("Mean intensity")
    ax3.grid()
    ax3.set_xlim(data_bin[key].min(),data_bin[key].max())

    # Title and fname
    ax0.set_title(
        f"%04d - %04d %s = %s"
        % (im_ids[0], im_ids[-1], key, np.round(data_bin[key].values[i], 3))
    )
    fname = "SAXS_ImId_%04d_%04d_%03d_%s.png" % (im_ids[0], im_ids[-1], i, USER)

    # Save
    fname = path.join(folder_gif, fname)
    variable_images_1d.append(fname)
    plt.savefig(fname)
    plt.close()

# Create gif for 1d AI
if len(im_ids) > 1:
    fname = f"SAXS_ImId_%04d_%04d_%s.gif" % (im_ids[0], im_ids[-1], USER)
else:
    fname = f"SAXS_ImId_%04d_%s.gif" % (im_ids[0], USER)

#var = [imageio.imread(file) for file in variable_images_1d]
gif_path = path.join(folder_general, fname)
print("Saving gif:%s" % gif_path)
helper.create_gif(variable_images_1d,gif_path,fps=3)
#imageio.mimsave(gif_path, var, fps=2)
print("Done!")

In [ ]:
# Drop images
data_bin_save = data_bin.drop_vars(["images"])

# Save log
folder = join(folder_general, "Logs")
helper.create_folder(folder)
fname = join(folder, "SAXS_Log_ImId_%04d_%s.nc" % (im_id, USER))

print(f"Saving:", fname)
data_bin_save.to_netcdf(fname)